# 공공데이터포털에서 아파트 실거래가 수집

- 공공데이터포털(https://www.data.go.kr/index.do)에서 '아파트 실거래가'로 검색합니다.
- '오픈 API' 탭을 선택한 다음, **'아파트매매 실거래자료'**에 대해 활용신청합니다.
- 활용신청을 정상적으로 마쳤다면 **'API 승인키'**를 발급받게 됩니다.

## 0. 패키지 호출

In [ ]:
# 웹 크롤링에 필요한 패키지를 호출합니다.
from urllib import parse
from requests import get
from bs4 import BeautifulSoup as bts
import pandas as pd
import time

## 1. HTTP Request 실행

- 5자리 지역코드(시군구), 6자리 거래년월 및 API 승인키로 데이터를 받을 수 있습니다.
- API 승인키는 '아파트매매 실거래자료' 상세 설명 페이지에서 복사할 수 있습니다.
- 지역코드는 행정표준코드관리시스템(https://www.code.go.kr)에서 찾습니다.
- 거래년월은 관심있는 기간을 스스로 설정하면 됩니다.

In [ ]:
# API 승인키를 지정합니다. (따옴표 안에 자신의 API 승인키를 붙여넣으세요!)
key = ''

In [ ]:
# URL 디코딩 결과를 출력합니다.
parse.unquote(key)

In [ ]:
# 아파트매매 실거래가의 URL을 복사해서 붙입니다.
main_url = 'http://openapi.molit.go.kr:8081'
sub_path = '/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade'

In [ ]:
# 검색조건을 Query Strings로 생성합니다. (서울 강남구, 2020년 6월 거래건)
# API 승인키에 '%'가 포함되어 있으면 사전에 URL 디코딩을 실행해야 합니다.
params = {'LAWD_CD': '11680', 
          'DEAL_YMD': '202006', 
          'serviceKey': parse.unquote(key)}

In [ ]:
# HTTP Request를 실행합니다.
res = get(main_url + sub_path, params = params)

## 2. HTTP Response 확인

In [ ]:
# 응답 객체의 상태코드를 확인합니다. (200이면 정상!)
res.status_code

In [ ]:
# 응답 헤더를 확인합니다. (콘텐트 타입, 인코딩 방식 등)
res.headers

In [ ]:
# 요청 URL을 확인합니다. (퍼센트 더블 인코딩 발생 여부 확인!)
res.url

In [ ]:
# 응답 객체의 콘텐트를 출력합니다. (인코딩 방식을 지정해주어야 한글이 제대로 보입니다.)
res.content.decode('UTF-8')

## 3. XML을 데이터프레임으로 변환

In [ ]:
# XML 형태로 읽습니다.
xml = bts(res.content, 'lxml-xml')
xml

In [ ]:
# 데이터가 반복되는 'item' 노드를 리스트 객체로 생성합니다.
items = xml.select('item')
items

In [ ]:
# `items` 객체의 길이를 확인합니다.
len(items)

In [ ]:
# `items` 객체의 첫 번째 원소를 출력하여 형태를 확인합니다.
items[0]

In [ ]:
# pretty type으로 출력합니다.
print(items[0].prettify())

In [ ]:
# 'node'의 텍스트가 None인 경우, 에러가 발생하므로 사용자 정의 함수로 방지합니다.
def getItemText(xml, node):
    xmlText = xml.find(node)
    if xmlText == None:
        listText = ''
    else:
        listText = str(xmlText.text).strip()
    return listText

In [ ]:
# 빈 리스트 객체를 생성합니다.
rows = []

# 반복문을 실행하여 각 `item`의 노드에 포함된 데이터를 rows에 추가합니다.
for item in items:
    price = getItemText(item, '거래금액')
    byear = getItemText(item, '건축년도')
    acode = getItemText(item, '지역코드')
    bdong = getItemText(item, '법정동')
    jibun = getItemText(item, '지번')
    aptNm = getItemText(item, '아파트')
    tyear = getItemText(item, '년')
    tmnth = getItemText(item, '월').zfill(2)
    tdate = getItemText(item, '일').zfill(2)
    exUse = getItemText(item, '전용면적')
    floor = getItemText(item, '층')
    
    rows.append({'price': price,
                 'byear': byear,
                 'acode': acode,
                 'bdong': bdong,
                 'jibun': jibun,
                 'aptNm': aptNm,
                 'tyear': tyear,
                 'tmnth': tmnth,
                 'tdate': tdate,
                 'exUse': exUse,
                 'floor': floor})

In [ ]:
# `rows`의 크기를 확인합니다.
len(rows)

In [ ]:
# `rows`의 일부를 출력하여 형태를 확인합니다.
rows[:3]

In [ ]:
# `rows`를 데이터프레임으로 변환합니다.
df = pd.DataFrame(rows)

In [ ]:
# 처음 10행만 출력합니다.
df.head(10)

## 4. 데이터프레임 전처리

In [ ]:
# 컬럼별 자료형을 확인합니다.
df.dtypes

In [ ]:
# `price`(거래금액) 컬럼에서 콤마를 삭제한 다음, 정수형으로 변환합니다.
df['price'] = df['price'].str.replace(',', '').astype(int)

# 단위를 '만원'에서 '억원'으로 변경합니다.
df['price'] = df['price'] / 10000

In [ ]:
# `exUse`(전용면적) 컬럼을 실수형으로 변환합니다.
df['exUse'] = df['exUse'].astype(float)

In [ ]:
# 컬럼별 자료형을 재확인합니다.
df.dtypes

## 5. 사용자 정의 함수 생성

In [ ]:
# '지역코드'와 '거래년월'을 입력하면 데이터프레임으로 반환하는 사용자 정의 함수입니다.
def getAptData(area_cd, deal_ymd):
    
    main_url = 'http://openapi.molit.go.kr:8081'
    sub_path = '/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade'
    params = {'LAWD_CD': area_cd, 
              'DEAL_YMD': deal_ymd, 
              'serviceKey': parse.unquote(key)}
    
    res = get(main_url + sub_path, params = params)
    
    if res.status_code == 200:
        
        xml = bts(res.content, 'xml')
        items = xml.select('items > item')
        
        rows = []
        
        for item in items:
            price = getItemText(item, '거래금액')
            byear = getItemText(item, '건축년도')
            acode = getItemText(item, '지역코드')
            bdong = getItemText(item, '법정동')
            jibun = getItemText(item, '지번')
            aptNm = getItemText(item, '아파트')
            tyear = getItemText(item, '년')
            tmnth = getItemText(item, '월').zfill(2)
            tdate = getItemText(item, '일').zfill(2)
            exUse = getItemText(item, '전용면적')
            floor = getItemText(item, '층')
            
            rows.append({'price': price,
                         'byear': byear,
                         'acode': acode,
                         'bdong': bdong,
                         'jibun': jibun,
                         'aptNm': aptNm,
                         'tyear': tyear,
                         'tmnth': tmnth,
                         'tdate': tdate,
                         'exUse': exUse,
                         'floor': floor})
    
        df = pd.DataFrame(rows)
        
        df['price'] = df['price'].str.replace(',', '').astype(int)
        df['price'] = df['price'] / 10000
        
        df['exUse'] = df['exUse'].astype(float)
        
    else:
        df = pd.DataFrame([])
    
    time.sleep(1)
    
    return df

In [ ]:
# 사용자 정의 함수를 테스트합니다.
df = getAptData(area_cd = '11680', deal_ymd = '202001')

# 처음 10행만 출력합니다.
df.head(10)

In [ ]:
# 2020년 1~6월을 리스트 객체로 생성합니다.
ymds = ['202001', '202002', '202003', '202004', '202005', '202006']

In [ ]:
# 최종 저장 객체인 `apt`를 빈 데이터프레임으로 생성합니다.
apt = pd.DataFrame([])

# 반복문을 실행합니다.
for ymd in ymds:
    df = getAptData(area_cd = '11680', deal_ymd = ymd)
    apt = pd.concat([apt, df])

In [ ]:
# `apt` 객체의 정보를 확인합니다.
apt.info()

In [ ]:
# 처음 10행만 출력합니다.
apt.head(10)

In [ ]:
# `trade`(거래일자) 컬럼을 생성합니다.
apt['trade'] = apt['tyear'] + apt['tmnth'] + apt['tdate']
apt['trade']

## 6. 다양한 그래프 그리기

In [ ]:
# 필요한 패키지를 호출합니다.
import numpy as np
from matplotlib import pyplot as plt
from dfply import *

# 그래프에서 한글이 제대로 출력되도록 폰트를 지정합니다.
plt.rc('font', family = 'AppleGothic')

In [ ]:
# `price`(거래금액) 컬럼의 기술통계량을 확인합니다.
apt['price'].describe()

In [ ]:
# `price`(거래금액) 컬럼으로 도수분포표를 생성합니다.
bins = np.arange(0, 61, 5)
hist, bins = np.histogram(apt['price'], bins)

# 빈도수를 출력합니다.
hist

In [ ]:
# `price`(거래금액) 컬럼으로 히스토그램을 그립니다.
plt.figure(figsize = (6, 4), dpi = 100)
plt.hist(apt['price'], 
         bins,
         color = 'gray',
         ec = 'black',
         density = True)
plt.title('서울 강남구 아파트 거래금액(2020년 1~6월)', fontsize = 14)
plt.xlabel('거래금액(억원)', fontsize = 12)
plt.show()

In [ ]:
# `price`(거래금액) 컬럼으로 상자수염그림을 그립니다.
plt.figure(figsize = (6, 4), dpi = 100)
plt.boxplot(apt['price'], 
            sym = 'red', 
            boxprops = dict(c = 'blue', lw = 2),
            whiskerprops = dict(c = 'red', ls = '--'))
plt.title('서울 강남구 아파트 거래금액(2020년 1~6월)', fontsize = 14)
plt.xticks([1], ['거래금액(억원)'])
plt.show()

In [ ]:
# `exUse`(전용면적)과 `price`(거래금액) 간 산점도를 그립니다.
plt.figure(figsize = (6, 4), dpi = 100)
plt.scatter(apt['exUse'], apt['price'], c = 'gray')
plt.title('전용면적과 거래금액의 관계', fontsize = 14)
plt.axvline(apt['exUse'].mean(), c = 'b', ls = '--', lw = 1)
plt.axhline(apt['price'].mean(), c = 'b', ls = '--', lw = 1)

high = apt[apt['price'] >= 50]
plt.scatter(high['exUse'], high['price'], c = 'red')

for x, y, label in zip(high['exUse'], high['price'], high['aptNm']):
    plt.text(x - 5, y, label, 
             ha = 'right', va = 'center', c = 'darkblue')

plt.show()

In [ ]:
# `price`(거래금액)이 50억 이상인 행을 확인합니다.
high

In [ ]:
# `trade`(거래일자)별 거래건수를 생성합니다.
tradeCnt = apt >> group_by(X.trade) >> summarise(count = X.trade.count())
tradeCnt

In [ ]:
# `tradeCnt`의 `trade`와 `count` 컬럼으로 선그래프를 그립니다.
plt.figure(figsize = (6, 4), dpi = 100)
plt.plot(tradeCnt['trade'], 
         tradeCnt['count'], 
         color = 'darkred', 
         marker = 'o',
         linestyle = '-', 
         linewidth = 1)
plt.title('일별 아파트 거래건수 현황', fontsize = 14)
plt.xlabel('거래일자', fontsize = 12)
plt.ylabel('거래건수', fontsize = 12)
plt.show()

In [ ]:
# `bdong`(법정동)별 거래건수를 생성합니다.
bdongCnt = apt >> group_by(X.bdong) >> summarise(count = X.bdong.count())
bdongCnt

In [ ]:
# `bdongCnt`의 `bdong`과 `count` 컬럼으로 막대그래프를 그립니다.
plt.figure(figsize = (6, 4), dpi = 100)
plt.bar(bdongCnt['bdong'], bdongCnt['count'], color = 'orange')
plt.title('법정동별 아파트 거래건수 현황', fontsize = 14)
plt.xticks(rotation = 90)
plt.xlabel('법정동명', fontsize = 12)
plt.ylabel('거래건수', fontsize = 12)

plt.ylim(0, 250)
for x, y in zip(bdongCnt['bdong'], bdongCnt['count']):
    plt.text(x, y + 0.01, y, 
             ha = 'center', va = 'bottom', c = 'k')

plt.show()

In [ ]:
# `bdong`(법정동)별 평균 거래금액을 생성합니다.
priceMean = (apt >> 
             group_by(X.bdong) >> 
             summarise(pmean = X.price.mean()) >>
             mutate(pmean = np.round(X.pmean, 1)))
priceMean

In [ ]:
# `priceMean`의 `bdong`과 `pmean` 컬럼으로 가로 막대그래프를 그립니다.
plt.figure(figsize = (6, 4), dpi = 100)
plt.barh(priceMean['bdong'], priceMean['pmean'], color = 'skyblue')
plt.gca().invert_yaxis()
plt.title('법정동별 평균 거래금액 현황', fontsize = 14)
plt.xlabel('평균금액', fontsize = 12)
plt.ylabel('법정동명', fontsize = 12)

plt.xlim(0, 30)
for x, y in zip(priceMean['pmean'], priceMean['bdong']):
    plt.text(x + 0.01, y, x, 
             ha = 'left', va = 'center', c = 'k')

plt.show()

In [ ]:
# `exUse`(전용면적)당 `price`(거래금액) 컬럼을 생성합니다.
apt['unitp'] = apt['price'] / apt['exUse']

In [ ]:
# `bdong`(법정동)별 평균 전용면적당 거래금액을 생성합니다.
unitpMean = (apt >> 
             group_by(X.bdong) >> 
             summarise(unitp = X.unitp.mean()) >> 
             mutate(unitp = np.round(X.unitp, 3)))
unitpMean

In [ ]:
# `unitpMean`의 `bdong`과 `unitp` 컬럼으로 가로 막대그래프를 그립니다.
plt.figure(figsize = (6, 4), dpi = 100)
plt.barh(unitpMean['bdong'], unitpMean['unitp'], color = 'royalblue')
plt.gca().invert_yaxis()
plt.title('법정동별 전용면적당 평균 거래금액 현황', fontsize = 14)
plt.xlabel('전용면적당 평균금액', fontsize = 12)
plt.ylabel('법정동명', fontsize = 12)

plt.xlim(0, 0.32)
for x, y in zip(unitpMean['unitp'], unitpMean['bdong']):
    plt.text(x + 0.01, y, x, 
             ha = 'left', va = 'center', c = 'k')

plt.show()

## End of Document